# Autograph: Graphs for complex code

In this notebook, we'll explore various scenarios from the lesson on `Creating graphs for complex code`. We'll delve into practical examples to deepen our understanding of how complex code structures can be represented and analyzed effectively using computational graphs.

## Imports

In [ ]:
import tensorflow as tf

Seemingly simple functions can sometimes pose challenges when converting them into graph mode. Fortunately, Autograph helps by automatically generating the complex graph code required for us. For example, consider a function that performs basic operations like multiplication and addition—Autograph efficiently handles these operations, ensuring they are optimized for performance within TensorFlow's graph execution environment. This automation streamlines the development process, allowing us to focus more on designing the logic rather than the intricacies of graph compatibility.

In [ ]:
# Define a TensorFlow variables 'a' and `b` initialized with the float values 1.0 and 2.0 respectively
a = tf.Variable(1.0)
b = tf.Variable(2.0)

# This decorator converts the function 'f' below into a TensorFlow graph function for optimized execution.
@tf.function
def f(x,y):
    a.assign(y * b)
    b.assign_add(x * a)
    return a + b

print(f(1.0, 2.0))

# This converts the TensorFlow graph function 'f' back into readable Python code and prints it.
print(tf.autograph.to_code(f.python_function))

tf.Tensor(10.0, shape=(), dtype=float32)
def tf__f(x, y):
    with ag__.FunctionScope('f', 'fscope', ag__.ConversionOptions(recursive=True, user_requested=True, optional_features=(), internal_convert_user_code=True)) as fscope:
        do_return = False
        retval_ = ag__.UndefinedReturnValue()
        ag__.converted_call(ag__.ld(a).assign, (ag__.ld(y) * ag__.ld(b),), None, fscope)
        ag__.converted_call(ag__.ld(b).assign_add, (ag__.ld(x) * ag__.ld(a),), None, fscope)
        try:
            do_return = True
            retval_ = ag__.ld(a) + ag__.ld(b)
        except:
            do_return = False
            raise
        return fscope.ret(retval_, do_return)



Here's a function that evaluates whether the sign of a number is positive. This straightforward check is a good example of how simple logical operations can be efficiently handled, even when integrating them into larger, more complex graph-based computations.

In [ ]:
@tf.function
def sign(x):
    if x > 0:
        return 'Positive'
    else:
        return 'Negative or zero'

print("Sign = {}".format(sign(tf.constant(2))))
print("Sign = {}".format(sign(tf.constant(-2))))

print(tf.autograph.to_code(sign.python_function))

Sign = b'Positive'
Sign = b'Negative or zero'
def tf__sign(x):
    with ag__.FunctionScope('sign', 'fscope', ag__.ConversionOptions(recursive=True, user_requested=True, optional_features=(), internal_convert_user_code=True)) as fscope:
        do_return = False
        retval_ = ag__.UndefinedReturnValue()

        def get_state():
            return (do_return, retval_)

        def set_state(vars_):
            nonlocal retval_, do_return
            (do_return, retval_) = vars_

        def if_body():
            nonlocal retval_, do_return
            try:
                do_return = True
                retval_ = 'Positive'
            except:
                do_return = False
                raise

        def else_body():
            nonlocal retval_, do_return
            try:
                do_return = True
                retval_ = 'Negative or zero'
            except:
                do_return = False
                raise
        ag__.if_stmt(ag__.ld(x) > 0, if_body, else

Below is another function that incorporates a while loop, showcasing how control flow structures, like loops, can be effectively managed within a computational graph. This ability is particularly useful for creating dynamic and iterative processes in graph mode.

In [ ]:
@tf.function
def f(x):
    while tf.reduce_sum(x) > 1:
        tf.print(x)
        x = tf.tanh(x)
    return x

print(tf.autograph.to_code(f.python_function))

def tf__f(x):
    with ag__.FunctionScope('f', 'fscope', ag__.ConversionOptions(recursive=True, user_requested=True, optional_features=(), internal_convert_user_code=True)) as fscope:
        do_return = False
        retval_ = ag__.UndefinedReturnValue()

        def get_state():
            return (x,)

        def set_state(vars_):
            nonlocal x
            (x,) = vars_

        def loop_body():
            nonlocal x
            ag__.converted_call(ag__.ld(tf).print, (ag__.ld(x),), None, fscope)
            x = ag__.converted_call(ag__.ld(tf).tanh, (ag__.ld(x),), None, fscope)

        def loop_test():
            return ag__.converted_call(ag__.ld(tf).reduce_sum, (ag__.ld(x),), None, fscope) > 1
        ag__.while_stmt(loop_test, loop_body, get_state, set_state, ('x',), {})
        try:
            do_return = True
            retval_ = ag__.ld(x)
        except:
            do_return = False
            raise
        return fscope.ret(retval_, do_return)



The next function below integrates a for loop and an if statement, illustrating how TensorFlow can handle common control structures within its graph-based computations, enhancing the flexibility and expressiveness of model definitions.

In [ ]:
@tf.function
def sum_even(items):
    s = 0
    for c in items:
        if c % 2 > 0:
            continue
        s += c
    return s

print(tf.autograph.to_code(sum_even.python_function))

def tf__sum_even(items):
    with ag__.FunctionScope('sum_even', 'fscope', ag__.ConversionOptions(recursive=True, user_requested=True, optional_features=(), internal_convert_user_code=True)) as fscope:
        do_return = False
        retval_ = ag__.UndefinedReturnValue()
        s = 0

        def get_state_2():
            return (s,)

        def set_state_2(vars_):
            nonlocal s
            (s,) = vars_

        def loop_body(itr):
            nonlocal s
            c = itr
            continue_ = False

            def get_state():
                return (continue_,)

            def set_state(vars_):
                nonlocal continue_
                (continue_,) = vars_

            def if_body():
                nonlocal continue_
                continue_ = True

            def else_body():
                nonlocal continue_
                pass
            ag__.if_stmt(ag__.ld(c) % 2 > 0, if_body, else_body, get_state, set_state, ('continue_',), 1)

            def

## Print statements

Tracing behavior can differ significantly in graph mode. For example, consider a function that prints the value of its input parameter. When this function is called repeatedly within a loop, such as calling `f(2)` five times and then `f(3)`, the output would typically display each call's input. However, when this function is decorated with `@tf.function` and converted to graph mode, TensorFlow optimizes the computation and might trace the function fewer times than it's called, reflecting different execution dynamics.

In [ ]:
def f(x):
    print("Traced with", x)

for i in range(5):
    f(2)

f(3)

Traced with 2
Traced with 2
Traced with 2
Traced with 2
Traced with 2
Traced with 3


When we decorate a function with `@tf.function` and execute it, especially one that includes a print statement like `f(2)` being called repeatedly in a loop, we might notice that the print statement only appears once for `f(2)`. This happens because TensorFlow's `@tf.function` decorator optimizes the function by converting it into a graph, and during this process, redundant calls are minimized. This optimization can lead to the function's side effects, such as printing, only occurring during the first execution of a unique call, demonstrating how TensorFlow handles graph execution efficiency.

In [ ]:
@tf.function
def f(x):
    print("Traced with", x)

for i in range(5):
    f(2)

f(3)

Traced with 2
Traced with 3


If we compare `print` to `tf.print` in the context of TensorFlow's graph mode, `tf.print` stands out because it is graph-aware. Unlike the standard `print` function, which might not execute as expected within TensorFlow's graph-optimized loops, `tf.print` will execute each time it is called within a loop in graph mode.


If we run code that includes both `print` and `tf.print` in a loop, and the function is decorated with `@tf.function`, we will notice that `tf.print` will output each time the loop iterates, faithfully reflecting the loop's dynamics. In contrast, `print` may only execute once due to TensorFlow's graph optimization processes. This difference highlights the importance of using `tf.print` for debugging and output tasks within TensorFlow's advanced execution environments.

In [ ]:
@tf.function
def f(x):
    print("Traced with", x)
    # added tf.print
    tf.print("Executed with", x)

for i in range(5):
    f(2)

f(3)

Traced with 2
Executed with 2
Executed with 2
Executed with 2
Executed with 2
Executed with 2
Traced with 3
Executed with 3


## Avoid defining variables inside the function

Below is a undecorated function which defines a tensor `v` and adds the input `x` to it. In this case, it executes as a regular Python function.

In [ ]:
def f(x):
    v = tf.Variable(1.0)
    v.assign_add(x)
    return v

print(f(1))

<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=2.0>


When we decorate the function with `@tf.function` and it includes a `tf.Variable` definition within it, an error will occur upon execution. This happens because, in TensorFlow's graph mode, variables should be defined outside of the function. This setup ensures that variables are created only once and then utilized throughout subsequent function calls. Graph mode functions are meant to consist of operations or transformations applied to these predefined variables or inputs, rather than defining new variables during each function call. This design helps maintain consistency and efficiency in TensorFlow’s optimized, graph-based computational model.

In [ ]:
@tf.function
def f(x):
    v = tf.Variable(1.0)
    v.assign_add(x)
    return v

print(f(1))

ValueError: in user code:

    File "<ipython-input-10-5729586b3383>", line 3, in f  *
        v = tf.Variable(1.0)

    ValueError: tf.function only supports singleton tf.Variables created on the first call. Make sure the tf.Variable is only created once or created outside tf.function. See https://www.tensorflow.org/guide/function#creating_tfvariables for more information.


To resolve this error, we should move the line `v = tf.Variable(1.0)` to the top of the cell, positioning it before the `@tf.function` decorator. By doing this, the variable `v` is initialized only once and exists outside the TensorFlow function, ready to be utilized within the decorated function. This adjustment aligns with TensorFlow's requirement that variables be instantiated outside the scope of graph-executed functions, ensuring they are not repeatedly recreated during function calls. This setup enhances performance and avoids potential issues with state management in graph mode.

In [ ]:
# Define the variables outside of the decorated function
v = tf.Variable(1.0)

@tf.function
def f(x):
    return v.assign_add(x)

print(f(5))

tf.Tensor(6.0, shape=(), dtype=float32)
